<a href="https://colab.research.google.com/github/rutvi/cs50Lecture0/blob/master/nisarg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers


In [2]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from keras.layers import Input, Dense, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dropout, Flatten, Activation, Lambda
from keras.models import Model
from keras import backend as K
from keras.optimizers import Adam
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd

# Load BERT tokenizer and model
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

ModuleNotFoundError: ignored

In [ ]:
# Load dataset
dataset = pd.read_csv('/content/drive/MyDrive/M.E. sem4/PrePostQuestionsbreakup-1.csv')

# Convert dataset to DataFrame
df = pd.DataFrame(dataset)

In [ ]:
df.head()

In [ ]:
# Tokenize input text

encoded_data  = df['question'].apply(lambda x: tokenizer.encode_plus(x, max_length=128, truncation=True, padding='max_length',
                                add_special_tokens=True, return_tensors='tf'))


In [ ]:
# Create TensorFlow dataset from encoded data
input_ids = np.array([encoded_data[i]['input_ids'] for i in range(len(encoded_data))]).squeeze()
attention_mask = np.array([encoded_data[i]['attention_mask'] for i in range(len(encoded_data))]).squeeze()
token_type_ids = np.array([encoded_data[i]['token_type_ids'] for i in range(len(encoded_data))]).squeeze()
input_ids = tf.constant(input_ids)
attention_mask = tf.constant(attention_mask)
token_type_ids = tf.constant(token_type_ids)

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((input_ids, attention_mask, token_type_ids))

In [ ]:
# Define a function to convert a batch of inputs to a batch of embeddings
@tf.function
def generate_embeddings(input_ids, attention_mask, token_type_ids):
    embeddings = bert_model([input_ids, attention_mask, token_type_ids])[0]
    return embeddings


In [1]:
# Generate embeddings for each batch
batch_size = 4
batches = dataset.batch(batch_size)
embeddings = []
for i,batch in batches:
    batch_input_ids, batch_attention_mask, batch_token_type_ids = batch
    batch_embeddings = generate_embeddings(batch_input_ids, batch_attention_mask, batch_token_type_ids)
    embeddings.append(batch_embeddings)
    # Append the embeddings to the output file
    if i == 0:
        np.save('/content/drive/MyDrive/M.E. sem4/n_embeddings.npy', embeddings.numpy())
    else:
        with open('n_embeddings.npy', 'ab') as f:
            np.save(f, embeddings.numpy())

    # Clear the embeddings array to free up RAM
    del embeddings
    tf.keras.backend.clear_session()



NameError: ignored